# GIS module

Les opérations GIS font partie intégrantes des opérations réalisées en hydrologie. Cette page 

In [1]:
import geopandas as gpd
import pandas as pd
import warnings
import leafmap
import numpy as np
import xarray as xr

from IPython.core.display import HTML
from pathlib import Path

import xdatasets as xd
import xhydro.gis as xhgis
from xhydro.indicators import get_yearly_op

## Creating a map

In [2]:
m = leafmap.Map(center=(48.63, -74.71), 
                zoom=5,
                basemap="USGS Hydrography")


## Watershed delineation

### a) From a list of coordinates

In [3]:
lng_lat = [(-69.81971, 48.14720), # Lac Saint-Jean watershed
           (-74.393438, 45.572442) # Ottawa river watershed
          ]

### b) From markers on a map

![test](../../docs/_static/_images/example_draw_marker,png)

In [4]:
# The following code is only useful for the documentation. You should instead remove this code and
# interact with the map as shown above by positionning markers at sites of interest
m.draw_features = [{'type': 'Feature',
  'properties': {},
  'geometry': {'type': 'Point', 'coordinates': [-73.118597, 46.042467]}}] # Richelieu watershed

In [5]:
%%time

gdf = xhgis.watershed_delineation(coordinates=lng_lat,
                                  map=m)
gdf

HTML(value="<html>\n<body>\n  <div class='my-legend'>\n  <div class='legend-title'>Upstream Area (sq. km).</di…

CPU times: user 4.07 s, sys: 652 ms, total: 4.72 s
Wall time: 4.65 s


,HYBAS_ID,Upstream Area (sq. km).,geometry,category,color
0,7120034330,87595.8,"POLYGON ((-74.37864 48.88141, -74.37452 48.886...",3,#41b6c4
1,7120398781,144026.8,"POLYGON ((-80.07991 46.77860, -80.08529 46.782...",5,#081d58
2,7120382860,23717.7,"POLYGON ((-73.77437 43.36757, -73.77557 43.388...",1,#ffffd9


In [6]:
m.zoom_to_gdf(gdf)
HTML(m.to_html())

### c) From [xdatasets](https://github.com/hydrologie/xdatasets) (Not implemented yet)

This functionality fetches a list of basins from [xdatasets](https://github.com/hydrologie/xdatasets)' supported datasets, and upon request, [xdatasets](https://github.com/hydrologie/xdatasets) provides a `gpd.GeoDataFrame` containing the precalculated boundaries for these basins. The user is encouraged to use official watershed boundaries if they exist rather than creating new ones.

In [7]:
gdf = xd.Query(
    **{
        "datasets": {
            "deh_polygons": {
                "id": ["031101","0421*"],
                "regulated": ["Natural"],
            }
        }
    }
).data.reset_index()

gdf

,Station,Superficie,geometry
0,031101,111.713104,"POLYGON ((-73.98519 45.21072, -73.98795 45.209..."
1,042102,623.479187,"POLYGON ((-78.57120 46.70742, -78.57112 46.707..."
2,042103,579.479614,"POLYGON ((-78.49014 46.64514, -78.49010 46.645..."


## Extract geographical watershed properties

a) Let's first extract watershed properties

In [8]:
xhgis.watershed_properties(gdf)

,Station,Superficie,area,perimeter,gravelius,centroid
0,031101,111.713104,1.117131e+08,65169.772478,1.739359,"(-74.14353174932663, 45.17637767020061)"
1,042102,623.479187,6.234792e+08,291795.718170,3.296575,"(-78.36569866204955, 46.47218010707938)"
2,042103,579.479614,5.794796e+08,283765.058390,3.325331,"(-78.37036445281987, 46.48287117609677)"


In [9]:
xhgis.watershed_properties(gdf[['Station', 'geometry']],
                           unique_id='Station',
                           output_format='xarray')

<xarray.Dataset>
Dimensions:    (Station: 3)
Coordinates:
  * Station    (Station) object '031101' '042102' '042103'
Data variables:
    area       (Station) float64 1.117e+08 6.235e+08 5.795e+08
    perimeter  (Station) float64 6.517e+04 2.918e+05 2.838e+05
    gravelius  (Station) float64 1.739 3.297 3.325
    centroid   (Station) object (-74.14353174932663, 45.17637767020061) ... (...

b) Let's then extract climate indicators

In [10]:
datasets = {
    "era5_land_reanalysis": {"variables": ["t2m", "tp", "sd"]},
}
space = {
    "clip": "polygon",  # bbox, point or polygon
    "averaging": True,
    "geometry": gdf,  # 3 polygons
    "unique_id": "Station",
}
time = {
    "start": "1981-01-01",
    "end": "2010-12-31",
    "timezone": "America/Montreal",
}

ds = xd.Query(datasets=datasets, space=space, time=time).data.squeeze()

Spatial operations: processing polygon 042103 with era5_land_reanalysis: : 3it [00:00,  4.72it/s]


Because the next few steps use [xclim](https://xclim.readthedocs.io/en/stable/index.html) under the hood, the dataset is required to be CF-compliant. At a minimum, the xarray.DataArray used has to follow these principles:

- The dataset needs a time dimension.
- If there is a spatial dimension, such as Station in the example below, it needs an attribute cf_role with timeseries_id as its value.
- The variable will at the very least need a units attribute, although other attributes such as long_name and cell_methods are also expected by xclim and warnings will be generated if they are missing.

In [11]:
ds['tas'] = ds.t2m - 273.15 # We should also consider using xarray-pint/xclim to improve units conversion 
ds["tas"].attrs.update(ds.t2m.attrs)
ds["tas"].attrs.update({"units": "C", "cell_methods": "time: mean"})
ds["tp"].attrs.update({"cell_methods": "time: mean within days"})
ds["sd"].attrs.update({"units": "m", "cell_methods": "time: mean within days"})
ds

<xarray.Dataset>
Dimensions:  (time: 262968, Station: 3)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 ... 2010-12-31T23:00:00
  * Station  (Station) object '031101' '042102' '042103'
    source   <U20 'era5_land_reanalysis'
Data variables:
    t2m      (Station, time) float64 251.6 251.0 251.3 ... 275.6 275.5 275.5
    tp       (Station, time) float64 1.505e-08 6.174e-08 ... 3.698e-06 6.662e-07
    sd       (Station, time) float64 0.03733 0.03733 0.03733 ... 0.06421 0.06384
    tas      (Station, time) float64 -21.59 -22.17 -21.9 ... 2.455 2.389 2.339

In [12]:
timeargs = {
    "01": {"month": [1]},
    "02": {"month": [2]},
    "03": {"month": [3]},
    "04": {"month": [4]},
    "05": {"month": [5]},
    "06": {"month": [6]},
    "07": {"month": [7]},
    "08": {"month": [8]},
    "09": {"month": [9]},
    "10": {"month": [10]},
    "11": {"month": [11]},
    "12": {"month": [12]},
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer_fall": {"date_bounds": ["06-20", "11-19"]},
    "year": {"date_bounds": ["01-01", "12-31"]},
    }

operations = {
    "tas": ["max","mean", "min"],
    "tp": ["sum"], 
    "sd": ["mean"], 
}


In [13]:
ds_climatology = xr.merge(
    [
        get_yearly_op(ds, input_var=variable, op=op, timeargs=timeargs)
        for (variable, ops) in operations.items() for op in ops
    ]
)
ds_climatology

<xarray.Dataset>
Dimensions:               (Station: 3, time: 30)
Coordinates:
  * Station               (Station) object '031101' '042102' '042103'
    source                <U20 'era5_land_reanalysis'
  * time                  (time) datetime64[ns] 1981-01-01 ... 2010-01-01
Data variables: (12/75)
    tas_max_01            (Station, time) float64 6.34 4.833 ... -4.901 3.161
    tas_max_02            (Station, time) float64 14.46 4.733 ... 4.389 3.544
    tas_max_03            (Station, time) float64 16.96 14.31 ... 8.564 12.78
    tas_max_04            (Station, time) float64 21.76 21.52 ... 24.34 25.98
    tas_max_05            (Station, time) float64 26.1 29.42 ... 24.25 32.24
    tas_max_06            (Station, time) float64 29.13 28.11 ... 30.66 25.42
    ...                    ...
    sd_mean_10            (Station, time) float64 0.0001856 ... 0.0004031
    sd_mean_11            (Station, time) float64 0.002668 ... 0.007307
    sd_mean_12            (Station, time) float64 0.02428 0.003355 ... 0.05267
    sd_mean_spring        (Station, time) float64 0.003797 0.0567 ... 0.03331
    sd_mean_summer_fall   (Station, time) float64 0.000253 ... 0.0007276
    sd_mean_year          (Station, time) float64 0.008973 0.02961 ... 0.02575
Attributes:
    cat:variable:          ('tas_max_01',)
    cat:xrfreq:            AS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [14]:
pd.set_option('display.max_rows', 100)
ds_climatology.mean('time').to_dataframe().T

Station,031101,042102,042103
source,era5_land_reanalysis,era5_land_reanalysis,era5_land_reanalysis
tas_max_01,6.965602,3.642321,3.640586
tas_max_02,7.062366,4.035679,4.044133
tas_max_03,14.727131,10.508251,10.525447
tas_max_04,22.04756,20.00788,19.973493
tas_max_05,27.27062,25.861504,25.819365
tas_max_06,30.193029,28.86869,28.824476
tas_max_07,31.172714,29.498319,29.448428
tas_max_08,30.302,28.586009,28.537646
tas_max_09,27.867745,25.764821,25.724547
